In [10]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import (DataLoader,)  # Gives easier dataset managment and creates mini batches
#import torchvision  # torch package for vision related things
#import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
#import torchvision.transforms as transforms
import pandas as pd
import random
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import RandomizedSearchCV
from tqdm.auto import tqdm
from transformers import DistilBertTokenizer
from torchsummary import summary

# Data Loading
Grabbing the dataset with the like count included

In [11]:
# Split the downloaded Youtube01-Psy.csv file into train, validation and test sets
data = pd.read_csv('SpamDataset/actual_train.csv')
# data = pd.read_csv('TSwift_ShakeItOff_Spam.csv')
data = data.sample(frac=1).reset_index(drop=True)  # Shuffle the data
train_data = data[:int(0.85*len(data))]
val_data = data[int(0.85*len(data)):]

# Tokenize the data using distillbert
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Turn pd dataframe into a tokenized PyTorch dataset that DataLoader can use via the SpamDataset class
class SpamDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        comment_tokens = row['CONTENT']
        like_count = row['LIKE_COUNT']
        summary = row['SUMMARY']
        label = row['CLASS']
        return comment_tokens, like_count, summary, label
    
    def collate_fn(self, batch):
        comments = [row[0] for row in batch]
        like_counts = [row[1] for row in batch]
        summary = [row[2] for row in batch]
        labels = [row[3] for row in batch]
        comment_tokens = tokenizer.batch_encode_plus(comments, truncation=True, padding='max_length', max_length=70, return_tensors='pt')
        summary_tokens = tokenizer.batch_encode_plus(summary, truncation=True, padding='max_length', max_length=70, return_tensors='pt')
        return comment_tokens, torch.tensor(like_counts), summary_tokens, torch.tensor(labels)
    
train_dataset = SpamDataset(train_data)
val_dataset = SpamDataset(val_data)


# Load Data and collate it
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=train_dataset.collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=val_dataset.collate_fn)

In [12]:
# Print a random sample from the train_loader to confirm correct loading and tokenization
sample = next(iter(train_loader))
print(tokenizer.decode(sample[0]['input_ids'][0]))
print(tokenizer.decode(sample[2]['input_ids'][0]))

[CLS] this comment will randomly get lot's of likes and replies for no reason. i also like jello. strawberry jello. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] katy perry's " roar " empowers with its anthemic chorus and vibrant visuals. the music video depicts perry surviving a jungle plane crash, transforming into a jungle queen who finds her strength. with its message of resilience and self - empowerment, " roar " resonated globally, becoming one of perry's most iconic [SEP]


# Model
- Model 1 - include Like count
- Model 2 - include LLM evaluation of the video 

In [13]:
## The network includes an Embedding layer, an Attention layer, an LSTM layer and a Linear layer
class CombinedLSTM(nn.Module):
    def __init__(self, embedding_dict, embedding_size, hidden_size, num_layers, num_classes, dropout):
        super(CombinedLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(embedding_dict, embedding_size)
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.attention = nn.Linear(hidden_size, 1)


        self.embedding_summary = nn.Embedding(embedding_dict, embedding_size)
        self.dropout_summary = nn.Dropout(dropout)
        self.lstm_summary = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.attention_summary = nn.Linear(hidden_size, 1)


        self.fc = nn.Linear(2 * hidden_size + 1, num_classes) # +1 for like count
        
    def forward(self, comment_sequence, like_count, summary_sequence):
        # Evaluate the comment sequence
        # comment_sequence: (batch_size, max_seq_length)
        comment_embedding = self.dropout(self.embedding(comment_sequence))
        # embedding shape: (batch_size, max_seq_length, embedding_size)
        output_comments, (h_n, c_n) = self.lstm(comment_embedding)
        # output shape: (batch_size, max_seq_length, hidden_size)
        # h_n shape: (num_layers, batch_size, hidden_size)
        attention_weights = F.softmax(self.attention(output_comments), dim=1)
        # attention_weights shape: (batch_size, max_seq_length, 1)
        output_comments = torch.sum(attention_weights * output_comments, dim=1)

        # Evaluate the summary sequence
        # summary_sequence: (batch_size, max_seq_length)
        summary_embedding = self.dropout_summary(self.embedding_summary(summary_sequence))
        # embedding shape: (batch_size, max_seq_length, embedding_size)
        output_summary, (h_n, c_n) = self.lstm_summary(summary_embedding)
        # output shape: (batch_size, max_seq_length, hidden_size)
        # h_n shape: (num_layers, batch_size, hidden_size)
        attention_weights = F.softmax(self.attention_summary(output_summary), dim=1)
        # attention_weights shape: (batch_size, max_seq_length, 1)
        output_summary = torch.sum(attention_weights * output_summary, dim=1)

        # Combine the comment and summary feature outputs with the like count
        # out shape: (batch_size, hidden_size)
        out = torch.cat((output_comments, output_summary, like_count.unsqueeze(1)), 1)
        # out shape: (batch_size, hidden_size + 1)
        out = self.fc(out)
        # out shape: (batch_size, num_classes)
        return out

# Training

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
# Set the hyperparameters for all the models     # TODO: analyze these by model
embedding_dict = tokenizer.vocab_size
embedding_size = 128
hidden_size = 256
num_layers = 4
num_classes = 2
dropout = 0.4
learning_rate = 0.0007
num_epochs = 50

# Initialize the model, loss function and optimizer
model = CombinedLSTM(embedding_dict, embedding_size, hidden_size, num_layers, num_classes, dropout).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_loader):
        comment_tokens, like_counts, summary_tokens, labels = batch
        comment_sequence = comment_tokens['input_ids'].to(device)
        like_counts = like_counts.to(device)
        summary_sequence = summary_tokens['input_ids'].to(device)
        labels = labels.to(device)
        outputs = model(comment_sequence, like_counts, summary_sequence)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

  0%|          | 0/29 [00:00<?, ?it/s]

# Validation

In [ ]:
# Validation
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    predictions = []
    targets = []
    model.eval()

    with torch.no_grad():
        for comments, like_counts, summary, target in loader:
            comments = {key: value.to(device) for key, value in comments.items()}
            like_counts = like_counts.to(device)
            summary = {key: value.to(device) for key, value in summary.items()}
            target = target.to(device)

            scores = model(comments['input_ids'], like_counts, summary['input_ids'])
            _, predicted = scores.max(1)
            # Store decoded original comments, model predictions and actual labels 
            num_correct += (predicted == target).sum()
            num_samples += predicted.size(0)

            predictions.extend(predicted.tolist())
            targets.extend(target.tolist())

    # model.train()
    accuracy = num_correct/num_samples
    return accuracy, predictions, targets

def results(model):
    train_acc, train_preds, train_targets = check_accuracy(train_loader, model)
    val_acc, val_preds, val_targets = check_accuracy(val_loader, model)

    print(f'Train accuracy: {train_acc}')
    print(f'Validation accuracy: {val_acc}')

    train_cm = confusion_matrix(train_targets, train_preds)
    val_cm = confusion_matrix(val_targets, val_preds)

    print(f'Train Confusion Matrix:\n{train_cm}')
    print(f'Validation Confusion Matrix:\n{val_cm}')


    # Classification report
    train_report = classification_report(train_targets, train_preds)
    val_report = classification_report(val_targets, val_preds)

    print(f'Train Classification Report:\n{train_report}')
    print(f'Validation Classification Report:\n{val_report}')

In [ ]:
results(model)

Train accuracy: 1.0
Validation accuracy: 0.9478527903556824
Train Confusion Matrix:
[[997   0]
 [  0 846]]
Validation Confusion Matrix:
[[182   1]
 [ 16 127]]
Train Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       997
           1       1.00      1.00      1.00       846

    accuracy                           1.00      1843
   macro avg       1.00      1.00      1.00      1843
weighted avg       1.00      1.00      1.00      1843

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.99      0.96       183
           1       0.99      0.89      0.94       143

    accuracy                           0.95       326
   macro avg       0.96      0.94      0.95       326
weighted avg       0.95      0.95      0.95       326



In [ ]:
from torchviz import make_dot
embedding_dict = tokenizer.vocab_size
embedding_size = 128
hidden_size = 256
num_layers = 1
num_classes = 2
dropout = 0.4
learning_rate = 0.0007
num_epochs = 15

# Initialize the model, loss function and optimizer
model = CombinedLSTM(embedding_dict, embedding_size, hidden_size, num_layers, num_classes, dropout).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

batch = next(iter(train_loader))
scores = model(batch[0]['input_ids'], batch[1], batch[2]['input_ids'])

print(model)
make_dot(scores, params=dict(list(model.named_parameters()))).render("model_images/feature_combination_lstm", format="png")

c:\Users\a_sta\anaconda3\Lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


CombinedLSTM(
  (embedding): Embedding(30522, 128)
  (dropout): Dropout(p=0.4, inplace=False)
  (lstm): LSTM(128, 256, batch_first=True, dropout=0.4)
  (attention): Linear(in_features=256, out_features=1, bias=True)
  (embedding_summary): Embedding(30522, 128)
  (dropout_summary): Dropout(p=0.4, inplace=False)
  (lstm_summary): LSTM(128, 256, batch_first=True, dropout=0.4)
  (attention_summary): Linear(in_features=256, out_features=1, bias=True)
  (fc): Linear(in_features=513, out_features=2, bias=True)
)


'model_images\\feature_combination_lstm.png'